## Part1: Think about the data

- Why are you interested in this data set?
- Clearly state if/how the data was pre-processed (Was the largest connected com-
ponent extracted? Was a sample of vertices or edges taken? If so, describe the sampling
process that was used.)?
- Before doing any analysis, answer the question. What characteristics do you expect
the vertices with high centrality values to have and why? Specifically, think about non-graph
characteristics. For example, in a graph where nodes represent cities and edges are roads
between them, we might expect highly central cities to have high populations or to house
major industries.

## Part2: Write Python code for graph analysis

In [57]:
# libraries
import networkx as nx
import numpy as np

# 2.1 Function to determine number of vertices in a graph
def number_of_vertices(graph):
    # tuples to list
    my_list = [item for x in graph for item in x]
    # remove duplicates
    unique_set = set(my_list)
    counter = len(unique_set)
    return counter


# 2.2 Function for finding the degree of a vertex
def degrees_of_vertex(graph, vertex):
    # Converting tuples to list
    my_list = [item for x in graph for item in x]
    counter = 0
    for i in my_list:
        if i == vertex:
            counter = counter + 1
    return counter

# Helper function: Get number of edges from subgraph
def subgraph_edges(graph, vertices):
    G = nx.Graph()
    G.add_edges_from(graph)
    induced_subgraph = G.subgraph(vertices)
    number_of_edges = nx.number_of_edges(induced_subgraph)
    return number_of_edges


# Helper function: Generate edges_list in graph
def generate_graph(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

# Function for finding the clustering coefficient of a vertex
# ref: https://www.geeksforgeeks.org/python-find-the-tuples-containing-the-given-element-from-a-list-of-tuples/1
def clustering_coefficient(graph, vertex):
    # Obtains a list of all the neighbors of the given vertex
    filtered_list = list(filter(lambda x: vertex in x, graph))
    # Converts the list of tuples into a regular list
    regular_list = [item for x in filtered_list for item in x]
    # Remove the vertex value from the regular list to get the list of just the neighbors
    neighbors = list(filter(vertex.__ne__, regular_list))
    # Calls a helper function that generates a graph from the edge list and creates a subgraph
    # of neighbors and then finds the number of edges amongst the subgraph of neighbors
    total_edges_actual = subgraph_edges(neighbors, graph)
    # Gets the number of neighbors
    number_of_neighbors = len(neighbors)
    # Calculates the number of total edges possible by (n-1)*(n/2) where n is the number of vertices
    total_edges_possible = (number_of_neighbors - 1) * (number_of_neighbors / 2)
    # Calculates the clustering coefficient by dividing
    # the number of actual edges by the total number of possible edges
    clustering_coefficient = total_edges_actual / total_edges_possible
    return clustering_coefficient

# Function for finding the betweenness centrality of a vertex
def betweenness_centrality(edgelist, vertex):
    values = []
    betweenness = 0
    size = number_of_vertices(edgelist)
    # Generate a graph from the given edgelist using a helper function
    graph = generate_graph(edgelist)

    # To determine if edgelist starts with 0 vs 1
    if edgelist[0][0] == 1:
        vertex_list = list(range(1, 1 + size))
    else:
        vertex_list = list(range(0, size))

    # Removes the specified vertex from the vertex list since it
    # won't be used in making a list of all possible vertex pairs
    vertex_list.remove(vertex)

    # Create a list of all the vertex pairs excluding x,x pairs (i.e. 1,1 or 2,2 etc.)
    for s in range(len(vertex_list)):
        for t in range(len(vertex_list)):
            if s != t:
                values = values + [[vertex_list[s], vertex_list[t]]]

    # Get rid of mirrored duplicates (i.e. [[0,1],[1,0]] -> [[0,1]])
    s = set()
    out = []
    for i in values:
        t = tuple(i)
        if t in s or tuple(reversed(t)) in s:
            continue
        s.add(t)
        out.append(i)

    # Extract the columns from the nested list so we have a list of every
    # possible node pair with no repeats and no x, x node pairs
    x_bar = [i[0] for i in out]
    y_bar = [i[1] for i in out]

    # Simultaneously iterate through the separate columns representing all vertex pairs and find all shortest paths
    for (x, y) in zip(x_bar, y_bar):
        count = 0

        # Get a list of all the shortest paths for every pair of nodes
        list_shortest_paths = list([p for p in nx.all_shortest_paths(graph, source=x, target=y)])

        # Get the total number of shortest paths
        number_of_shortest_path = len(list_shortest_paths)

        # Flatten the nested list into a list of single elements
        flat_list = [item for sublist in list_shortest_paths for item in sublist]

        # Search for the number of occurrences of the betweenness node
        for i in flat_list:
            if i == vertex:
                count = count + 1

        # Calculate betweenness centrality
        betweenness = betweenness + (count / number_of_shortest_path)
    return betweenness

# Function for creating an adjacency matrix
def adjacency_matrix(edgelist):
    # Get the size of the edgelist (number of nodes)
    nodes = num_vertices(edgelist)
    # Create an array of all zeros based on the number of nodes
    matrix = np.zeros((nodes, nodes))
    # Use the vertices as coordinate to iterate thru the array
    x_bar = [i[0] for i in edgelist]
    y_bar = [i[1] for i in edgelist]

    # Using the coordinates change a zero to a one
    for (x, y) in zip(x_bar, y_bar):
        matrix[x][y] = 1
        matrix[y][x] = 1
    return matrix



# def prestiege_centrality(matrix_arrr):
#     arr = np.ones((matrix_arrr.shape[1],1))
#     result = np.dot(matrix_arrr, arr)
#     return result


In [2]:
# Test case graphs for Project 2: CSCI 347

g0 = nx.Graph()
g0.add_edge(1,2)
g0.add_edge(2,3)
g0.add_edge(2,4)
g0.add_edge(3,5)
g0.add_edge(4,5)
g0.add_edge(5,6)

graph_0 = [(1,2), (2,3), (2,4), (3,5),(4,5),(5,6)]

graph_1 = [(1,2), (2,3), (3,4), (4,5),(5,1)]
g1 = nx.Graph()
for edge in graph_1:
    g1.add_edge(edge[0], edge[1])

graph_2 = [(1,2),(1,3), (2,3),(2,4) ,(3,4), (4,5),(5,1)]
g2 = nx.Graph()
for edge in graph_2:
    g2.add_edge(edge[0], edge[1])

graph_3 = [(1,2),(1,3), (2,3),(2,4) ,(3,4),(3,6), (4,5),(5,1),(6,7),(6,8), (7,8), (8,9) ]
g3 = nx.Graph()
for edge in graph_3:
    g3.add_edge(edge[0], edge[1])

g4 = nx.path_graph(50)
graph_4 = []
for line in nx.generate_edgelist(g4, data=False):
    vi = int(line.split()[0])
    vj = int(line.split()[1])
    pair = (vi, vj)
    graph_4.append(pair)

g5 = nx.complete_graph(7)
graph_5 = []
for line in nx.generate_edgelist(g5, data=False):
    vi = int(line.split()[0])
    vj = int(line.split()[1])
    pair = (vi, vj)
    graph_5.append(pair)

g6 = nx.cycle_graph(8)
graph_6 = []
for line in nx.generate_edgelist(g6, data=False):
    vi = int(line.split()[0])
    vj = int(line.split()[1])
    pair = (vi, vj)
    graph_6.append(pair)

n=200 # number of nodes for Barabasi-Albert graph
q=4 # number of edges each new node has in the generation process of BA graph

g7 = nx.barabasi_albert_graph(n, q, seed=34)
graph_7 = []
for line in nx.generate_edgelist(g7, data=False):
    vi = int(line.split()[0])
    vj = int(line.split()[1])
    pair = (vi, vj)
    graph_7.append(pair)


g8 = nx.erdos_renyi_graph(100,0.13, seed=15)
graph_8 = []
for edge in g8.edges():
    graph_8.append(edge)



In [60]:
# number_of_vertices(graph_8)
# degrees_of_vertex(graph_7, 27) 
# graph_7.degree(27)
G = nx.Graph()
# G.add_edge(graph_7[0])
gg = graph_3
for i in range(number_of_vertices(gg)):
    G.add_edge(gg[i][0],gg[i][1])

# nx.betweenness_centrality(G)[23]

# degrees_of_vertex(graph_3, 6) 
# graph_7[0][1]
# graph_7[0]
# clustering_coefficient(graph_8, 46)

# nx.average_shortest_path_length(G)
# print(adjacency_matrix(graph_1))
# betweenness_centrality(graph_8, 39)
# nx.eigenvector_centrality(G)
# nx.pagerank(G)
# prestiege_centrality(adj_matrix(graph_1))
arr2 = adj_matrix(graph_1)
arr = np.ones((5,1))
# np.dot(arr2, arr)
arr2

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

## Part 3: Analyze the graph data